In [ ]:
# Load the dataset
import pandas as pd

url = 'https://raw.githubusercontent.com/rfordatascience/tidytuesday/master/data/2021/2021-02-02/telecom.csv'
df = pd.read_csv(url)

In [ ]:
# Explore the first few rows of the dataset
df.head()

In [ ]:
# Explore statistical summaries
df.describe()

In [ ]:
# Feature Engineering: Create 'total_spend' as the sum of day, evening, and night charges
df['total_spend'] = df['total_day_charge'] + df['total_eve_charge'] + df['total_night_charge']

# Select features and target variable
features = ['account_length', 'total_day_minutes', 'total_spend']
target = 'churned'

X = df[features]
y = df[target]

In [ ]:
# Split the data into training and test sets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Train a Logistic Regression model
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()
model.fit(X_train, y_train)

In [ ]:
# Make predictions and evaluate accuracy
y_pred = model.predict(X_test)
from sklearn.metrics import accuracy_score

print(f'Accuracy: {accuracy_score(y_test, y_pred):.2f}')

In [ ]:
# Optional: Additional evaluation metrics
from sklearn.metrics import confusion_matrix, classification_report

print('Confusion Matrix:')
print(confusion_matrix(y_test, y_pred))

print('\nClassification Report:')
print(classification_report(y_test, y_pred))